# Getting GPT-4 to write a UI to interact with itself

ChatGPT Plus allows users to upload images for analysis, but I am still on the wait list. Therefore, I had ChatGPT write me the code for a Streamlit app that would let me do this on my own.

I wrote the below code to interact with ChatGPT and then used the code to interact with ChatGPT, passing it the code itself and asking it to create a UI based off of it. You can see the contents of our conversation below, and the final resulting Streamlit app in `app.py`

In [1]:
from openai import OpenAI
from dotenv import load_dotenv

client = OpenAI()

In [50]:
import base64
import requests
import os

# Function to base-64 encode images
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

class ChatBot:
  
    def __init__(self, 
                 api_key=os.getenv('OPENAI_API_KEY'), 
                 model='gpt-4-vision-preview', 
                 system='You are a helpful assistant.') -> None:
        self._api_key = api_key
        self._model = model
        self._system = system

        self._messages = []
        self._max_tokens = 2000

    @property
    def _headers(self):
        return {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self._api_key}"
        }
    
    @property
    def _payload(self):
        return {
            "model": self._model,
            "messages": self.messages,
            "max_tokens": self._max_tokens
        }

    @property
    def messages(self):
       return self._messages
    
    def clear_messages(self):
       self._messages = []

    def _add_message(self, message):
       self._messages.append(message)

    def _ask_text(self, text):
        self._add_message(
           {"role": "user", "content": text},
        )                

        response = client.chat.completions.create(
            model=self._model,
            messages=self.messages,
            max_tokens=self._max_tokens
        )

        
        self._add_message({k: v for (k, v) in response.choices[0].message if k in ['role', 'content']})  

        return response.choices[0].message.content
    
    def _ask_image(self, text, image_path):
        self.messages.append({
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": text
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encode_image(image_path)}"
                }
                }
            ]
        })

        # get response
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=self._headers, json=self._payload)
        
        # add to chat history
        self._add_message(m := response.json()['choices'][0]['message'])

        return print(m['content'])
    
    def chat(self, text, image_path=None):
       if not image_path:
          return self._ask_text(text)
       else:
          return self._ask_image(text, image_path)


In [37]:
bot = ChatBot()

In [51]:
bot = ChatBot()
print(bot.chat("""
I wrote the following Python code to interact with the OpenAI GPT-4 API:
               
```python
import base64
import requests
import os

# Function to base-64 encode images
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

class ChatBot:
  
    def __init__(self, 
                 api_key=os.getenv('OPENAI_API_KEY'), 
                 model='gpt-4-vision-preview', 
                 system='You are a helpful assistant.') -> None:
        self._api_key = api_key
        self._model = model
        self._system = system

        self._messages = []
        self._max_tokens = 2000

    @property
    def _headers(self):
        return {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self._api_key}"
        }
    
    @property
    def _payload(self):
        return {
            "model": self._model,
            "messages": self.messages,
            "max_tokens": self._max_tokens
        }

    @property
    def messages(self):
       return self._messages
    
    def clear_messages(self):
       self._messages = []

    def _add_message(self, message):
       self._messages.append(message)

    def _ask_text(self, text):
        self._add_message(
           {"role": "user", "content": text},
        )                

        response = client.chat.completions.create(
            model=self._model,
            messages=self.messages,
            max_tokens=self._max_tokens
        )

        
        self._add_message({k: v for (k, v) in response.choices[0].message if k in ['role', 'content']})  

        return response.choices[0].message.content
    
    def _ask_image(self, text, image_path):
        self.messages.append({
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": text
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encode_image(image_path)}"
                }
                }
            ]
        })

        # get response
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=self._headers, json=self._payload)
        
        # add to chat history
        self._add_message(m := response.json()['choices'][0]['message'])

        return print(m['content'])
    
    def chat(self, text, image_path=None):
       if not image_path:
          return self._ask_text(text)
       else:
          return self._ask_image(text, image_path)
```

Write a Python streamlit app that looks like the attached image to allow users to use this code through a GUI
""".strip(),
image_path='ui.png'
))

To create a Streamlit app like the one in the provided image, you will need to write a Python script that uses the Streamlit library alongside your `ChatBot` class. Below is an example of how you can create this app. Run this script with Streamlit by saving it to a file (e.g., `app.py`) and then use the command `streamlit run app.py` in your terminal.

Please note that the code provided on its own won't work without the correct OpenAI API setup and permissions, as well as ensuring that the `requests` library is imported in the script.

```python
# Import Streamlit and other necessary libraries
import streamlit as st
from chat_bot import ChatBot  # Assuming your ChatBot class is in chat_bot.py
import tempfile

# Set up your Streamlit layout
st.title("ChatBot with Image Analysis")

# Sidebar for the app
st.sidebar.header("Upload an Image")

# User can upload an image
uploaded_file = st.sidebar.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

# Set up columns for input an

In [52]:
bot.chat("That's close, but the response from the model displays in the terminal rather than the streamlit GUI. Please fix this!")

'I understand the issue; the problem occurs because the `_ask_image` method of the `ChatBot` class uses `print` to display the response, which sends the output to the console instead of returning it to the Streamlit interface.\n\nTo fix this, you should modify the `_ask_image` method so that it returns the response directly instead of printing it. Here\'s the corrected version of the `_ask_image` method within the ChatBot class:\n\n```python\n# Inside ChatBot class, modify the _ask_image method\n\ndef _ask_image(self, text, image_path):\n    self.messages.append({\n        "role": "user",\n        "content": [\n            {"type": "text", "text": text},\n            {"type": "image_url",\n             "image_url": {\n                 "url": f"data:image/jpeg;base64,{encode_image(image_path)}"\n             }\n            }\n        ]\n    })\n\n    # get response\n    response = requests.post("https://api.openai.com/v1/chat/completions",\n                             headers=self._hea

In [55]:
print(bot.messages[-1]['content'])

I understand the issue; the problem occurs because the `_ask_image` method of the `ChatBot` class uses `print` to display the response, which sends the output to the console instead of returning it to the Streamlit interface.

To fix this, you should modify the `_ask_image` method so that it returns the response directly instead of printing it. Here's the corrected version of the `_ask_image` method within the ChatBot class:

```python
# Inside ChatBot class, modify the _ask_image method

def _ask_image(self, text, image_path):
    self.messages.append({
        "role": "user",
        "content": [
            {"type": "text", "text": text},
            {"type": "image_url",
             "image_url": {
                 "url": f"data:image/jpeg;base64,{encode_image(image_path)}"
             }
            }
        ]
    })

    # get response
    response = requests.post("https://api.openai.com/v1/chat/completions",
                             headers=self._headers, json=self._payload

In [56]:
print(bot.chat("That's good, but the uploaded image does not display in the streamlit GUI"))

My apologies for the oversight. This might be happening because the image path created from the temporary file might not be directly usable by Streamlit's `image` function due to differences in how temporary files are handled.

To make sure the uploaded image is displayed in the Streamlit GUI, you should read the image into memory and then display it using Streamlit's `image` function without relying on the file path. Here is how you can modify the Streamlit app script to display the uploaded image:

```python
import streamlit as st
from chat_bot import ChatBot  # Assuming your ChatBot class is in chat_bot.py
import io

# ... your existing Streamlit code ...

# Run the bot if the image is uploaded
if uploaded_file is not None:
    # To read the image into memory
    bytes_data = uploaded_file.read()
    image_stream = io.BytesIO(bytes_data)
    
    # Use the PIL library to open the image
    from PIL import Image
    image = Image.open(image_stream)

    # Display the uploaded image
 

In [57]:
print(bot.chat("Please use pillow instead of PIL"))

Certainly, you can use `Pillow` directly (which is imported as `PIL`, a historical artifact of when `Pillow` was originally forked from the PIL library). The following code demonstrates how you can display an uploaded image using the `Pillow` library in your Streamlit app:

```python
import streamlit as st
from chat_bot import ChatBot  # Assuming your ChatBot class is in chat_bot.py
from PIL import Image
import io

# ... your existing Streamlit code ...

# Run the bot if the image is uploaded
if uploaded_file is not None:
    # Read the image into a bytes object
    bytes_data = uploaded_file.getvalue()
    
    # Create a BytesIO object from the bytes data and use Pillow to open the image
    image = Image.open(io.BytesIO(bytes_data))

    # Display the uploaded image in the Streamlit app
    col1.header("Uploaded Image")
    col1.image(image, use_column_width=True)

    # Setup the input field and response area
    col2.header("Input")
    user_input = col2.text_input("Ask a question